In [1]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 15:53:35) [MSC v.1929 64 bit (AMD64)]
pandas version: 2.0.3
numpy version: 1.24.3
matplotlib version: 3.7.1
tqdm version: 4.65.0
sktime version: 0.20.1
xgboost version: 1.7.6
seaborn version: 0.12.2
scikit-learn version: 1.2.2
------------------------------------------------------------------------------


In [2]:
current_path = os.getcwd()
parent_path = os.path.abspath(os.path.join(current_path, '..', '..','..'))
sys.path.append(parent_path)
pd.set_option('display.max_columns', 30)
train = pd.read_csv('../../../../train.csv')
train.drop(['num_date_time'],axis=1,inplace=True)
test = pd.read_csv('../../../../test.csv')
test.drop(['num_date_time'],axis=1,inplace=True)
building = pd.read_csv('../../../../building_info.csv')
test = pd.read_csv('../../../../merge_test_encoding.csv', encoding = "CP949")
train_loc = pd.read_csv('../../../../train_location.csv')
train = pd.concat([train,train_loc['location']],axis=1)
def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [3]:
from preprocessing import fillnan
from preprocessing import preprocessing_all

train, test = fillnan(train, test)
train, test = preprocessing_all(train, test, building)

In [4]:
from preprocessing import preprocessing_all
from fe import new_fe
train, test = new_fe(train, test)
train

,num,date_time,temp,rainy,wind,hum,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,...,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,body_temp,low_power_day,power_diff_ratio,power_increase_summer,증가_day_hour_sum,감소_day_hour_sum,increase_all,decrease_all
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,0,781.2,0,0,0.0,0.0,0,0,12,0,1
1,1,20220608 00,19.0,0.0,3.2,61.0,1124.16,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,0,1159.0,0,0,0.0,0.0,0,0,12,0,1
2,1,20220615 00,17.7,0.0,3.3,72.0,1182.24,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,0,1274.4,0,0,0.0,0.0,0,0,12,0,1
3,1,20220622 00,24.1,0.0,2.5,80.0,1257.60,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,0,1928.0,0,1,0.0,0.0,0,0,12,0,1
4,1,20220629 00,25.7,0.8,3.5,89.0,1224.00,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,0,2287.3,0,2,0.0,0.0,0,0,12,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220726 23,25.9,0.0,0.1,90.0,610.32,9,57497.84,40035.23,0.0,0.0,0.0,23,1,...,0,0,0,0,1,2331.0,0,2,0.0,0.0,0,0,12,0,1
203996,100,20220802 23,26.9,0.0,1.0,90.0,655.68,9,57497.84,40035.23,0.0,0.0,0.0,23,1,...,0,0,0,0,1,2421.0,0,2,0.0,0.0,0,0,12,0,1
203997,100,20220809 23,27.8,0.0,2.2,82.0,650.40,9,57497.84,40035.23,0.0,0.0,0.0,23,1,...,0,0,0,0,1,2279.6,0,2,0.0,0.0,0,0,12,0,1
203998,100,20220816 23,24.5,0.0,0.1,98.0,669.12,9,57497.84,40035.23,0.0,0.0,0.0,23,1,...,0,0,0,0,1,2401.0,0,1,0.0,0.0,0,0,12,0,1


In [5]:
train['연면_냉방면적'] = train['연면적'] / (train['냉방면적'])
test['연면_냉방면적'] = test['연면적'] / (test['냉방면적'])
train['태양광효율'] = train['연면적'] / (train['태양광용량'] + 1)
test['태양광효율'] = test['연면적'] / (test['태양광용량'] + 1)
train['태양광효율'] = train['연면적'] / (train['태양광용량'] + 1)
test['태양광효율'] = test['연면적'] / (test['태양광용량'] + 1)

In [6]:
def assign_rainy_category(group):
    rainy_count = (group['rainy'] > 0).sum()
    if rainy_count == 0:
        return 0
    elif rainy_count > 0 and rainy_count <= 3:
        return 1
    elif rainy_count > 3 and rainy_count <= 9:
        return 2
    else:
        return 3

rainy_categories = train.groupby(['num', 'week', 'day']).apply(assign_rainy_category).reset_index(name='rainy_category')
train = pd.merge(train, rainy_categories, on=['num', 'week', 'day'], how='left')

rainy_categories_test = test.groupby(['num', 'week', 'day']).apply(assign_rainy_category).reset_index(name='rainy_category')
test = pd.merge(test, rainy_categories_test, on=['num', 'week', 'day'], how='left')
train = train.sort_values(['num','date_time'])
test = test.sort_values(['num','date_time'])

In [7]:
# 폭염 경보, 폭염 주의보, 열대야 피처를 초기화합니다.
train['heatwave_alert'] = 0
train['heatwave_caution'] = 0
train['tropical_night'] = 0

# 빌딩 ID별로 루프를 돕니다.
for building_id in train['num'].unique():
    # 해당 빌딩의 데이터만 선택
    building_data = train[train['num'] == building_id]

    # 년월일별로 루프를 돕니다.
    for date in building_data['date_time'].apply(lambda x: x.split(' ')[0]).unique():
        # 해당 빌딩, 날짜의 데이터만 선택
        daily_data = building_data[building_data['date_time'].str.startswith(date)]

        # 최대 온도 계산
        max_temp = daily_data['temp'].max()

        # 35도 이상이면 폭염 경보 1
        if max_temp >= 35:
            train.loc[(train['num'] == building_id) & (train['date_time'].str.startswith(date)), 'heatwave_alert'] = 1

        # 33도 이상이면 폭염 주의보 1
        if max_temp >= 33:
            train.loc[(train['num'] == building_id) & (train['date_time'].str.startswith(date)), 'heatwave_caution'] = 1

        # 모든 시간대의 온도가 25도 이상이면 열대야 1
        if daily_data['temp'].min() >= 25:
            train.loc[(train['num'] == building_id) & (train['date_time'].str.startswith(date)), 'tropical_night'] = 1


test['heatwave_alert'] = 0
test['heatwave_caution'] = 0
test['tropical_night'] = 0

# 빌딩 ID별로 루프를 돕니다.
for building_id in test['num'].unique():
    # 해당 빌딩의 데이터만 선택
    building_data = test[test['num'] == building_id]

    # 년월일별로 루프를 돕니다.
    for date in building_data['date_time'].apply(lambda x: x.split(' ')[0]).unique():
        # 해당 빌딩, 날짜의 데이터만 선택
        daily_data = building_data[building_data['date_time'].str.startswith(date)]

        # 최대 온도 계산
        max_temp = daily_data['temp'].max()

        # 35도 이상이면 폭염 경보 1
        if max_temp >= 35:
            test.loc[(test['num'] == building_id) & (test['date_time'].str.startswith(date)), 'heatwave_alert'] = 1

        # 33도 이상이면 폭염 주의보 1
        if max_temp >= 33:
            test.loc[(test['num'] == building_id) & (test['date_time'].str.startswith(date)), 'heatwave_caution'] = 1

        # 모든 시간대의 온도가 25도 이상이면 열대야 1
        if daily_data['temp'].min() >= 25:
            test.loc[(test['num'] == building_id) & (test['date_time'].str.startswith(date)), 'tropical_night'] = 1

In [8]:
train['temp_wind'] = train['temp'] / (train['wind']+1)
test['temp_wind'] = test['temp'] / (test['wind']+1)

In [9]:
from sklearn.preprocessing import LabelEncoder

# 원-핫 인코딩된 피처들
onehot_features = ['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소',
       '지식산업센터', '할인마트', '호텔및리조트']

# 원-핫 인코딩된 피처들에 대해 각 행에서 1의 값을 갖는 열의 이름을 얻음
train['building_type'] = train[onehot_features].idxmax(axis=1)
test['building_type'] = test[onehot_features].idxmax(axis=1)

# 라벨 인코더 생성
le = LabelEncoder()

# 'building_type' 열에 라벨 인코딩 적용
train['building_type'] = le.fit_transform(train['building_type'])
test['building_type'] = le.transform(test['building_type'])

# 원-핫 인코딩된 열 삭제
train = train.drop(columns=onehot_features)
test = test.drop(columns=onehot_features)


In [10]:
train.drop(['building_type'],axis=1,inplace=True)
test.drop(['building_type'],axis=1,inplace=True)

In [11]:
# train과 test에 각각 'is_train' 열 추가
train['is_train'] = True
test['is_train'] = False

# train과 test 데이터프레임을 병합
combined = pd.concat([train, test], ignore_index=True)

# 'num'과 'date_time'을 기준으로 정렬
combined = combined.sort_values(['num', 'date_time'])

# 온도 차이 계산 함수
def calculate_temp_diff(group):
    group['temp_diff'] = group['temp'].diff().fillna(0)
    group['tem_x_hum_diff'] = group['tem_x_hum'].diff().fillna(0)

    return group

# 'num' 열을 기준으로 그룹화하고 calculate_temp_diff 함수 적용
combined = combined.groupby('num').apply(calculate_temp_diff)

# 'is_train' 열을 기준으로 train과 test 데이터를 다시 나눔
train = combined[combined['is_train'] == True].drop(columns=['is_train'])
test = combined[combined['is_train'] == False].drop(columns=['is_train'])
# temp_diff 피처의 값이 양수면 1, 음수면 -1로 변경
train['temp_diff'] = train['temp_diff'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
test['temp_diff'] = test['temp_diff'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# tem_x_hum_diff 피처의 값이 양수면 1, 음수면 -1로 변경
train['tem_x_hum_diff'] = train['tem_x_hum_diff'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
test['tem_x_hum_diff'] = test['tem_x_hum_diff'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


# 필요한 경우 인덱스 재설정
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train

,num,date_time,temp,rainy,wind,hum,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,...,power_diff_ratio,power_increase_summer,증가_day_hour_sum,감소_day_hour_sum,increase_all,decrease_all,연면_냉방면적,태양광효율,rainy_category,heatwave_alert,heatwave_caution,tropical_night,temp_wind,temp_diff,tem_x_hum_diff
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0.0,0,0,12,0,1,2.795906,110634.00,0,0,0,0,9.789474,0,0
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,1,2,...,0.0,0,0,13,0,1,2.795906,110634.00,0,0,0,0,8.571429,-1,1
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,2,2,...,0.0,0,0,13,0,1,2.795906,110634.00,0,0,0,0,7.080000,-1,-1
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,3,2,...,0.0,0,0,13,0,1,2.795906,110634.00,0,0,0,0,6.958333,-1,1
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,4,2,...,0.0,0,8,5,0,0,2.795906,110634.00,0,0,0,0,4.842105,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,19,2,...,0.0,0,0,13,0,1,1.436181,57497.84,1,0,0,0,12.157895,-1,1
203996,100,20220824 20,22.4,0.0,1.3,86.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,20,2,...,0.0,0,0,13,0,1,1.436181,57497.84,1,0,0,0,9.739130,-1,-1
203997,100,20220824 21,21.3,0.0,1.0,92.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,21,2,...,0.0,0,1,12,0,0,1.436181,57497.84,1,0,0,0,10.650000,-1,1
203998,100,20220824 22,21.0,0.0,0.3,94.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,22,2,...,0.0,0,0,13,0,1,1.436181,57497.84,1,0,0,0,16.153846,-1,1


## XGB

In [12]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [13]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective_xgb(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_int('gamma', 1, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        #'alpha': trial.suggest_loguniform('alpha', 1e-3, 100.0),
        'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'random_state': 953
    }

    model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value = model.get_params()['alpha']
    model.set_params(**{'objective':weighted_mse(alpha_value)})
    model.fit(X_train, y_train, verbose = False, eval_set=[(X_val, y_val)], early_stopping_rounds=50)
    y_pred = model.predict(X_val)
    score = SMAPE(y_val, y_pred)

    return score


### 34 / 5 따로 fit

In [14]:
# 3, 4
preds = np.array([]) 
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0804_34.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.2, stratify=num_data['stratified_target'], random_state=953)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 3, 4이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([3, 4])]])

    # Validation set에서 day 값이 3, 4인 행만 남기기
    val_df = val_df[val_df['day'].isin([3, 4])]

    y_train = train_df['power']
    y_val = val_df['power']

    x_train, x_test = train_df.drop(['date_time'],axis=1), test.drop(['date_time'],axis=1)
    x_val = val_df.drop(['date_time'],axis=1)

    x_train.drop(['power'],axis=1,inplace=True)
    x_val.drop(['power'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=60)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    ##근데 저렇게 validation 하면 학습 셋도 좀 이상해지는데 이게 맞나?0.4
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    preds_34 = np.append(preds, y_pred)


[I 2023-08-03 22:23:56,491] A new study created in memory with name: no-name-99d8fe81-5131-4254-99fc-429d69abe192
C:\Users\ineeji\AppData\Local\Temp\ipykernel_46912\116189032.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\ineeji\AppData\Local\Temp\ipykernel_46912\116189032.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constr

In [15]:
# 5
preds = np.array([]) 
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0804_5.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.2, stratify=num_data['stratified_target'], random_state=953)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 5이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([5])]])

    # Validation set에서 day 값이 5인 행만 남기기
    val_df = val_df[val_df['day'].isin([5])]

    y_train = train_df['power']
    y_val = val_df['power']

    x_train, x_test = train_df.drop(['date_time'],axis=1), test.drop(['date_time'],axis=1)
    x_val = val_df.drop(['date_time'],axis=1)

    x_train.drop(['power'],axis=1,inplace=True)
    x_val.drop(['power'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=60)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=953)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    ##근데 저렇게 validation 하면 학습 셋도 좀 이상해지는데 이게 맞나?0.4
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    preds_5 = np.append(preds, y_pred)


[I 2023-08-04 04:25:06,594] A new study created in memory with name: no-name-0a88a614-6636-44b2-af5e-2d2d4cb3d67c
C:\Users\ineeji\AppData\Local\Temp\ipykernel_46912\116189032.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\ineeji\AppData\Local\Temp\ipykernel_46912\116189032.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constr

In [27]:
submission = pd.read_csv('../../../../sample_submission.csv')

# answer_34와 answer_5의 값을 새로운 컬럼에 할당
submission['answer_34'] = preds_34
submission['answer_5'] = preds_5

# 'num_date_time'에서 년월일 부분만 추출
submission['date'] = submission['num_date_time'].apply(lambda x: x.split('_')[1])
submission['date'] = submission['date'].apply(lambda x: x.split(' ')[0])

# 먼저 'answer' 컬럼을 'answer_34'와 'answer_5'의 평균으로 초기화
submission['answer'] = (submission['answer_34'] + submission['answer_5']) / 2

print(submission)
# 'date'가 '20220826' 또는 '20220827'인 행의 'answer' 값을 해당 날짜의 'answer_34' 값으로 변경
submission.loc[submission['date'].isin(['20220825', '20220826']), 'answer'] = submission.loc[submission['date'].isin(['20220825', '20220826']), 'answer_34']

# 'date'가 '20220828'인 행의 'answer' 값을 해당 날짜의 'answer_5' 값으로 변경
submission.loc[submission['date'] == '20220827', 'answer'] = submission.loc[submission['date'] == '20220827', 'answer_5']

# 불필요한 컬럼 삭제
submission = submission.drop(columns=['date', 'answer_34', 'answer_5'])

submission.to_csv('submit_v27_xgb_953_seperate_real_edit2.csv', index=False)
submission

         num_date_time       answer    answer_34     answer_5      date
0        1_20220825 00  1948.338623  1949.632446  1947.044800  20220825
1        1_20220825 01  1944.844604  1886.484741  2003.204468  20220825
2        1_20220825 02  1638.710571  1604.047119  1673.374023  20220825
3        1_20220825 03  1560.010986  1502.918579  1617.103394  20220825
4        1_20220825 04  1587.396484  1494.666382  1680.126587  20220825
...                ...          ...          ...          ...       ...
16795  100_20220831 19   913.898285   904.905579   922.890991  20220831
16796  100_20220831 20   808.115051   803.022827   813.207275  20220831
16797  100_20220831 21   742.103607   741.037842   743.169373  20220831
16798  100_20220831 22   661.415100   652.933899   669.896301  20220831
16799  100_20220831 23   553.260681   543.016785   563.504578  20220831

[16800 rows x 5 columns]


,num_date_time,answer
0,1_20220825 00,1949.632446
1,1_20220825 01,1886.484741
2,1_20220825 02,1604.047119
3,1_20220825 03,1502.918579
4,1_20220825 04,1494.666382
...,...,...
16795,100_20220831 19,913.898285
16796,100_20220831 20,808.115051
16797,100_20220831 21,742.103607
16798,100_20220831 22,661.415100


In [26]:
submission = pd.read_csv('../../../../sample_submission.csv')
submission['answer_34'] = preds_34
submission['answer_5'] = preds_5
# 'num_date_time'에서 년월일 부분만 추출
submission['date'] = submission['num_date_time'].apply(lambda x: x.split(' ')[1])
# 'date'가 '20220825' 또는 '20220827'인 행의 'power_5' 값을 0으로 변경
submission.loc[submission['date'].isin(['20220825', '20220826']), 'power_5'] = submission.loc[submission['date'].isin(['20220825', '20220826']), 'answer_34']
submission.loc[submission['date'].isin(['20220827']), 'answer_34'] = submission.loc[submission['date'].isin(['20220827']), 'power_5']
submission['answer'] = (submission['answer_34'] + submission['answer_5']) / 2
z = submission.copy()
submission = pd.read_csv('../../../../sample_submission.csv')
submission['answer'] = z['answer']
#submission.to_csv('submit_v25_xgb_81_seperate.csv', index = False)
submission

,num_date_time,answer
0,1_20220825 00,1959.369873
1,1_20220825 01,1932.714905
2,1_20220825 02,1676.093445
3,1_20220825 03,1548.757446
4,1_20220825 04,1560.819214
...,...,...
16795,100_20220831 19,899.233032
16796,100_20220831 20,808.955627
16797,100_20220831 21,757.248260
16798,100_20220831 22,656.413818


# AG

In [ ]:
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularDataset, TabularPredictor

mape_custom = make_scorer(name='SMAPE',
                                 score_func=SMAPE,
                                 optimum=0,
                                 greater_is_better=False)
train_data = TabularDataset(train)
test_data = TabularDataset(test)
predictor = TabularPredictor(label='power',  eval_metric=mape_custom).fit(train_data, presets='medium_quality',  ag_args_fit={'num_gpus': 0})

y_pred = predictor.predict(test_data)
predictor.leaderboard() #original FE

No path specified. Models will be saved in: "AutogluonModels\ag-20230731_094643\"
Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230731_094643\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   82.17 GB / 511.09 GB (16.1%)
Train Data Rows:    204000
Train Data Columns: 50
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the 

KeyboardInterrupt: 

In [20]:
predictor.feature_importance(train)

Computing feature importance via permutation shuffling for 44 features using 5000 rows with 5 shuffle sets...
	664.32s	= Expected runtime (132.86s per shuffle set)
	495.12s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
num_day_hour_mean,65.736601,1.187328,1.276576e-08,5,68.181326,63.291876
date_time,6.579000,0.303959,5.452113e-07,5,7.204855,5.953144
num_day_hour_std,5.889451,0.270891,5.355949e-07,5,6.447219,5.331682
연면적,2.414510,0.172040,3.072161e-06,5,2.768742,2.060278
num,1.949075,0.162155,5.696279e-06,5,2.282954,1.615195
데이터센터,1.596359,0.107343,2.438577e-06,5,1.817379,1.375338
냉방면적,1.528947,0.118810,4.340412e-06,5,1.773578,1.284316
CDH,1.522276,0.125423,5.480242e-06,5,1.780524,1.264028
temp,1.383953,0.154175,1.818010e-05,5,1.701400,1.066505
power_diff_ratio,1.176135,0.106621,8.016402e-06,5,1.395669,0.956601


# AG New_FE

In [62]:
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularDataset, TabularPredictor

mape_custom = make_scorer(name='SMAPE',
                                 score_func=SMAPE,
                                 optimum=0,
                                 greater_is_better=False)
train_data = TabularDataset(train)
test_data = TabularDataset(test)
predictor = TabularPredictor(label='power',  eval_metric=mape_custom).fit(train_data, presets='medium_quality',  ag_args_fit={'num_gpus': 0},excluded_model_types = ['CatBoost'])

y_pred = predictor.predict(test_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20230803_093913\"
Presets specified: ['medium_quality']


	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230803_093913\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   75.73 GB / 511.09 GB (14.8%)
Train Data Rows:    204000
Train Data Columns: 45
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regressi

[1000]	valid_set's l2: 22163.4	valid_set's SMAPE: -5.04824
[2000]	valid_set's l2: 17544	valid_set's SMAPE: -4.50106
[3000]	valid_set's l2: 15396.6	valid_set's SMAPE: -4.18558
[4000]	valid_set's l2: 14102.8	valid_set's SMAPE: -3.9867
[5000]	valid_set's l2: 13221.4	valid_set's SMAPE: -3.83654
[6000]	valid_set's l2: 12584.7	valid_set's SMAPE: -3.73558
[7000]	valid_set's l2: 12054.7	valid_set's SMAPE: -3.64422
[8000]	valid_set's l2: 11631.3	valid_set's SMAPE: -3.57147
[9000]	valid_set's l2: 11295.4	valid_set's SMAPE: -3.51519
[10000]	valid_set's l2: 11033.6	valid_set's SMAPE: -3.46164


	-3.461	 = Validation score   (-SMAPE)
	62.08s	 = Training   runtime
	0.38s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l2: 18355.4	valid_set's SMAPE: -4.43201
[2000]	valid_set's l2: 14394.5	valid_set's SMAPE: -3.99733
[3000]	valid_set's l2: 12632.8	valid_set's SMAPE: -3.74818
[4000]	valid_set's l2: 11838.8	valid_set's SMAPE: -3.59369
[5000]	valid_set's l2: 11302.2	valid_set's SMAPE: -3.48237
[6000]	valid_set's l2: 10817.3	valid_set's SMAPE: -3.38881
[7000]	valid_set's l2: 10443.5	valid_set's SMAPE: -3.30824
[8000]	valid_set's l2: 10150.6	valid_set's SMAPE: -3.243
[9000]	valid_set's l2: 9907.3	valid_set's SMAPE: -3.19868
[10000]	valid_set's l2: 9679.66	valid_set's SMAPE: -3.16302


	-3.1616	 = Validation score   (-SMAPE)
	58.33s	 = Training   runtime
	0.52s	 = Validation runtime
Fitting model: RandomForestMSE ...
		Input X contains infinity or a value too large for dtype('float32').
Detailed Traceback:
Traceback (most recent call last):
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1733, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1684, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, total_resources=total_resources, **model_fit_kwargs)
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\autogluon\core\models\abstract\abstract_model.py", line 829, in fit
    out = self._fit(**kwargs)
  File "c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\autogluon\tabular\models\rf\rf_m

[1000]	valid_set's l2: 13363.4	valid_set's SMAPE: -3.84008
[2000]	valid_set's l2: 10996.6	valid_set's SMAPE: -3.41487
[3000]	valid_set's l2: 10155.2	valid_set's SMAPE: -3.21652
[4000]	valid_set's l2: 9620.14	valid_set's SMAPE: -3.07579
[5000]	valid_set's l2: 9356.09	valid_set's SMAPE: -2.98966
[6000]	valid_set's l2: 9053.18	valid_set's SMAPE: -2.91974
[7000]	valid_set's l2: 8891.55	valid_set's SMAPE: -2.86755
[8000]	valid_set's l2: 8778.29	valid_set's SMAPE: -2.8289
[9000]	valid_set's l2: 8709.66	valid_set's SMAPE: -2.79353
[10000]	valid_set's l2: 8630.01	valid_set's SMAPE: -2.76402


	-2.7636	 = Validation score   (-SMAPE)
	79.26s	 = Training   runtime
	0.55s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-2.7551	 = Validation score   (-SMAPE)
	0.26s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 678.45s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230803_093913\")


In [64]:
predictor.leaderboard() #폭염경보 + 이전시간 온도차 + 빌딩 라벨인코딩으로

                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -2.755108       1.156371  195.707713                0.001241           0.260834            2       True          6
1        LightGBMLarge  -2.763562       0.551963   79.261877                0.551963          79.261877            1       True          5
2              XGBoost  -3.111982       0.603167  116.185003                0.603167         116.185003            1       True          4
3             LightGBM  -3.161621       0.517561   58.331221                0.517561          58.331221            1       True          2
4           LightGBMXT  -3.460959       0.380095   62.081198                0.380095          62.081198            1       True          1
5             CatBoost  -3.692553       0.009073  272.267638                0.009073         272.267638            1       True          3


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.755108,1.156371,195.707713,0.001241,0.260834,2,True,6
1,LightGBMLarge,-2.763562,0.551963,79.261877,0.551963,79.261877,1,True,5
2,XGBoost,-3.111982,0.603167,116.185003,0.603167,116.185003,1,True,4
3,LightGBM,-3.161621,0.517561,58.331221,0.517561,58.331221,1,True,2
4,LightGBMXT,-3.460959,0.380095,62.081198,0.380095,62.081198,1,True,1
5,CatBoost,-3.692553,0.009073,272.267638,0.009073,272.267638,1,True,3


In [52]:
predictor.leaderboard() #폭염경보 + 이전시간 온도차

                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -2.700228       1.301972  264.001806                0.000000           0.337093            2       True          6
1        LightGBMLarge  -2.716487       0.598792  121.342618                0.598792         121.342618            1       True          5
2              XGBoost  -3.064374       0.703180  142.322095                0.703180         142.322095            1       True          4
3             LightGBM  -3.068908       0.396613   66.402752                0.396613          66.402752            1       True          2
4           LightGBMXT  -3.326139       0.382917   58.443653                0.382917          58.443653            1       True          1
5             CatBoost  -3.636162       0.013451  432.787150                0.013451         432.787150            1       True          3


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.700228,1.301972,264.001806,0.000000,0.337093,2,True,6
1,LightGBMLarge,-2.716487,0.598792,121.342618,0.598792,121.342618,1,True,5
2,XGBoost,-3.064374,0.703180,142.322095,0.703180,142.322095,1,True,4
3,LightGBM,-3.068908,0.396613,66.402752,0.396613,66.402752,1,True,2
4,LightGBMXT,-3.326139,0.382917,58.443653,0.382917,58.443653,1,True,1
5,CatBoost,-3.636162,0.013451,432.787150,0.013451,432.787150,1,True,3


In [37]:
predictor.leaderboard() #폭염경보

                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -2.681919       1.149982  178.041445                0.000995           0.391893            2       True          6
1        LightGBMLarge  -2.702160       0.569348   82.720381                0.569348          82.720381            1       True          5
2              XGBoost  -2.999358       0.579639   94.929172                0.579639          94.929172            1       True          4
3             LightGBM  -3.091107       0.388938   64.011667                0.388938          64.011667            1       True          2
4           LightGBMXT  -3.358897       0.449513   67.408445                0.449513          67.408445            1       True          1
5             CatBoost  -3.614412       0.009530  209.346893                0.009530         209.346893            1       True          3


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.681919,1.149982,178.041445,0.000995,0.391893,2,True,6
1,LightGBMLarge,-2.702160,0.569348,82.720381,0.569348,82.720381,1,True,5
2,XGBoost,-2.999358,0.579639,94.929172,0.579639,94.929172,1,True,4
3,LightGBM,-3.091107,0.388938,64.011667,0.388938,64.011667,1,True,2
4,LightGBMXT,-3.358897,0.449513,67.408445,0.449513,67.408445,1,True,1
5,CatBoost,-3.614412,0.009530,209.346893,0.009530,209.346893,1,True,3


In [18]:
predictor.leaderboard() #fill na 내꺼

                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2  -2.705418       1.232151  200.417005                0.001001           0.364123            2       True          6
1        LightGBMLarge  -2.713969       0.599902   90.960732                0.599902          90.960732            1       True          5
2             LightGBM  -3.072975       0.404353   62.169089                0.404353          62.169089            1       True          2
3              XGBoost  -3.087154       0.631249  109.092149                0.631249         109.092149            1       True          4
4           LightGBMXT  -3.371737       0.607214   63.777059                0.607214          63.777059            1       True          1
5             CatBoost  -3.653967       0.006984  240.367055                0.006984         240.367055            1       True          3


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2.705418,1.232151,200.417005,0.001001,0.364123,2,True,6
1,LightGBMLarge,-2.713969,0.599902,90.960732,0.599902,90.960732,1,True,5
2,LightGBM,-3.072975,0.404353,62.169089,0.404353,62.169089,1,True,2
3,XGBoost,-3.087154,0.631249,109.092149,0.631249,109.092149,1,True,4
4,LightGBMXT,-3.371737,0.607214,63.777059,0.607214,63.777059,1,True,1
5,CatBoost,-3.653967,0.006984,240.367055,0.006984,240.367055,1,True,3


In [65]:
predictor.feature_importance(train)

Computing feature importance via permutation shuffling for 45 features using 5000 rows with 5 shuffle sets...
	703.51s	= Expected runtime (140.7s per shuffle set)
	528.65s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
num_day_hour_mean,69.230542,1.259845,1.315428e-08,5,71.824581,66.636504
date_time,6.498305,0.263896,3.256558e-07,5,7.041671,5.954939
num_day_hour_std,5.498241,0.226371,3.440264e-07,5,5.964343,5.032139
num,1.696980,0.176145,1.373215e-05,5,2.059664,1.334296
연면적,1.682921,0.135450,4.992308e-06,5,1.961814,1.404027
CDH,1.513424,0.145057,1.000439e-05,5,1.812098,1.214750
temp,1.437582,0.163732,1.984788e-05,5,1.774709,1.100455
power_diff_ratio,1.090795,0.099098,8.085433e-06,5,1.294839,0.886751
태양광효율,0.987542,0.084378,6.333697e-06,5,1.161277,0.813807
냉방면적,0.964913,0.099952,1.362076e-05,5,1.170715,0.759111


# ens

In [34]:
a = pd.read_csv('submission2.csv')
b = pd.read_csv('submit_v12_xgb_optuna_iter41_8888.csv')
c = a.copy()
c['answer'] = 0
c['answer'] = a['answer']*0.42185 + b['answer']*0.57815
#c['answer'] = (a['answer'] + b['answer']) / 2
c.to_csv('submit_v13_xgb_optuna_8512.csv',index=False)
c

,num_date_time,answer
0,1_20220825 00,2032.590085
1,1_20220825 01,2001.954761
2,1_20220825 02,1804.802206
3,1_20220825 03,1708.447136
4,1_20220825 04,1699.192685
...,...,...
16795,100_20220831 19,943.590352
16796,100_20220831 20,848.670576
16797,100_20220831 21,764.608321
16798,100_20220831 22,661.657046


In [81]:
a = pd.read_csv('11_12_xgb_41_720_8888_ens.csv')
a['answer'] = a['answer'] * 1.05
a.to_csv('sotapp.csv',index=False)
a

,num_date_time,answer
0,1_20220825 00,2201.706445
1,1_20220825 01,2157.875354
2,1_20220825 02,1942.385477
3,1_20220825 03,1818.851477
4,1_20220825 04,1827.483920
...,...,...
16795,100_20220831 19,923.047476
16796,100_20220831 20,850.975172
16797,100_20220831 21,780.236613
16798,100_20220831 22,650.756886


# 각 빌딩별 Validation값

In [123]:
import pandas as pd
import re

# 파일을 열어서 문자열을 읽어옵니다.
with open('best_params_values_0720v2.txt', 'r') as f:
    string = f.read()

# Extract the Iteration and Best Values
pattern = r"Iteration: (\d+),.*?Best Values: ([\d\.]+)"
matches = re.findall(pattern, string, re.DOTALL)

# Convert the matches into a DataFrame
df = pd.DataFrame(matches, columns=["Iteration", "Best Values"])
df["Iteration"] = df["Iteration"].astype(int)
df["Best Values"] = df["Best Values"].astype(float)

print(df)

    Iteration  Best Values
0           1     3.309706
1           2     6.319838
2           3     6.023522
3           4     3.278278
4           5     4.923040
..        ...          ...
95         96     2.608702
96         97     4.062261
97         98     6.098842
98         99     2.525203
99        100     4.073055

[100 rows x 2 columns]


In [124]:
info = pd.read_csv('building_info.csv')
concat = pd.concat([info,df['Best Values']],axis=1)
# Reorder the columns
cols = concat.columns.tolist()  # Convert the column names into a list
cols.insert(2, cols.pop(cols.index('Best Values')))  # Move 'Best Values' to the 3rd position (index 2)

concat = concat[cols]  # Reindex the DataFrame
concat

,건물번호,건물유형,Best Values,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,3.309706,110634.00,39570.00,-,-,-
1,2,건물기타,6.319838,122233.47,99000.00,-,-,-
2,3,건물기타,6.023522,171243.00,113950.00,40,-,-
3,4,건물기타,3.278278,74312.98,34419.62,60,-,-
4,5,건물기타,4.923040,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...,...
95,96,호텔및리조트,2.608702,93314.00,60500.00,-,-,-
96,97,호텔및리조트,4.062261,55144.67,25880.00,-,-,-
97,98,호텔및리조트,6.098842,53578.62,17373.75,-,-,-
98,99,호텔및리조트,2.525203,53499.00,40636.00,-,-,-


In [125]:
import pandas as pd
import re

# 파일을 열어서 문자열을 읽어옵니다.
with open('best_params_values_0721.txt', 'r') as f:
    string = f.read()

# Extract the Iteration and Best Values
pattern = r"Iteration: (\d+),.*?Best Values: ([\d\.]+)"
matches = re.findall(pattern, string, re.DOTALL)

# Convert the matches into a DataFrame
df = pd.DataFrame(matches, columns=["Iteration", "Best Values_day따로"])
df["Iteration"] = df["Iteration"].astype(int)
df["Best Values_day따로"] = df["Best Values_day따로"].astype(float)

print(df)

     Iteration  Best Values_day따로
0            1           3.958431
1            1           5.159142
2            2           7.236492
3            2           6.041059
4            3           5.471286
..         ...                ...
154         96           2.694907
155         97           3.705436
156         98           6.647394
157         99           2.348074
158        100           4.307969

[159 rows x 2 columns]


In [126]:
# Calculate the mean of 'Best Values' for each unique 'Iteration'
df_mean = df.groupby('Iteration')['Best Values_day따로'].mean().reset_index()
df_mean

,Iteration,Best Values_day따로
0,1,4.558786
1,2,6.638776
2,3,5.745692
3,4,3.593410
4,5,4.045336
...,...,...
95,96,2.694907
96,97,3.705436
97,98,6.647394
98,99,2.348074


In [127]:
concat = pd.concat([concat,df_mean['Best Values_day따로']],axis=1)
# Reorder the columns
cols = concat.columns.tolist()  # Convert the column names into a list
cols.insert(3, cols.pop(cols.index('Best Values_day따로')))  # Move 'Best Values' to the 3rd position (index 2)

concat = concat[cols]  # Reindex the DataFrame
concat

,건물번호,건물유형,Best Values,Best Values_day따로,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,3.309706,4.558786,110634.00,39570.00,-,-,-
1,2,건물기타,6.319838,6.638776,122233.47,99000.00,-,-,-
2,3,건물기타,6.023522,5.745692,171243.00,113950.00,40,-,-
3,4,건물기타,3.278278,3.593410,74312.98,34419.62,60,-,-
4,5,건물기타,4.923040,4.045336,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...,...,...
95,96,호텔및리조트,2.608702,2.694907,93314.00,60500.00,-,-,-
96,97,호텔및리조트,4.062261,3.705436,55144.67,25880.00,-,-,-
97,98,호텔및리조트,6.098842,6.647394,53578.62,17373.75,-,-,-
98,99,호텔및리조트,2.525203,2.348074,53499.00,40636.00,-,-,-


In [129]:
concat.to_csv('빌딩별_validation.csv',index=False, encoding = "CP949")